In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.core.pipeline import run_pipeline
from src.evaluation.regime import ic_by_regime


In [1]:
REGIONS = [
    "austin",
    "toronto",
    "vancouver",
    "new_york",
]


In [5]:
dfs = {r: run_pipeline(r) for r in REGIONS}


In [6]:
for r, df in dfs.items():
    assert {"score_xs", "fwd_return", "regime"}.issubset(df.columns)


In [7]:
rows = []
for r, df in dfs.items():
    df = df.copy()
    df["alpha"] = df["score_xs"]
    df["alpha_gated"] = (df["regime"] == "down").astype(float) * df["alpha"]
    out = ic_by_regime(df, signal_col="alpha_gated")
    out["region"] = r
    rows.append(out)

pd.concat(rows)


/Users/axl/Library/Python/3.12/lib/python/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/axl/Library/Python/3.12/lib/python/site-packages/numpy/lib/_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/Users/axl/Library/Python/3.12/lib/python/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/axl/Library/Python/3.12/lib/python/site-packages/numpy/lib/_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


,ic,positive_ratio,n_obs,region
down,-0.434744,0.337838,74.0,austin
up,NaN,0.000000,106.0,austin
down,-0.450909,0.546296,108.0,toronto
up,NaN,0.000000,72.0,toronto


In [8]:
summary = pd.concat(rows).reset_index()
summary


,index,ic,positive_ratio,n_obs,region
0,down,-0.434744,0.337838,74.0,austin
1,up,NaN,0.000000,106.0,austin
2,down,-0.450909,0.546296,108.0,toronto
3,up,NaN,0.000000,72.0,toronto


In [9]:
summary["abs_ic"] = summary["ic"].abs()
summary["is_tradable"] = (summary["ic"] < 0) & (summary["n_obs"] >= 50)
summary


,index,ic,positive_ratio,n_obs,region,abs_ic,is_tradable
0,down,-0.434744,0.337838,74.0,austin,0.434744,True
1,up,NaN,0.000000,106.0,austin,NaN,False
2,down,-0.450909,0.546296,108.0,toronto,0.450909,True
3,up,NaN,0.000000,72.0,toronto,NaN,False


In [10]:
(df["regime"] == "down").astype(float)


Series([], Name: regime, dtype: float64)

In [12]:
df["gate_soft"] = df["score_xs"].clip(upper=0)
df["alpha_soft"] = df["gate_soft"] * df["alpha"]


In [13]:
signal_col="alpha_soft"


In [15]:
df["gate_soft"] = df["score_xs"].clip(upper=0)
df["alpha_soft"] = df["gate_soft"] * df["alpha"]


In [16]:
out = ic_by_regime(df, signal_col="alpha_gated")


In [17]:
summary.groupby("index")[["ic", "abs_ic"]].mean()


,ic,abs_ic
index,,
down,-0.442826,0.442826
up,NaN,NaN


Affordability acts as a regime-dependent exposure scaler:
it suppresses alpha in expansionary regimes and amplifies it during affordability stress, consistently across regions.

In [18]:
summary.groupby("index")[["ic"]].count()


,ic
index,
down,2
up,0
